In [ ]:
!pip install -q tensorflow==2.8.0

import tensorflow as tf
print(tf.__version__)

     |███████████████████████▌        | 491.5 MB 1.2 MB/s eta 0:02:31

In [ ]:
!pip install -q transformers==4.18.0

import transformers
print(transformers.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/Code/anantha

/content/drive/MyDrive/Code/anantha


In [ ]:
PRE_TRAINED_MODEL = 'hasil-fine-tuning'

In [ ]:
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification

bert_tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL)
bert_model = TFBertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL, num_labels=6)

Some layers from the model checkpoint at hasil-fine-tuning were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at hasil-fine-tuning.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
LABEL_NAME = ['anger','disgust','fear','joy','sadness','surprise']
MAX_LEN = 80

In [ ]:
def get_emotion(text):
  predict_input = bert_tokenizer.encode(
    text,
    max_length=MAX_LEN,
    truncation=True,
    padding=True,
    return_tensors="tf")
  
  tf_output = bert_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1)
  label = tf.argmax(tf_prediction, axis=1)
  label = label.numpy()
  
  return LABEL_NAME[label[0]]

In [ ]:
import pandas as pd

fear_data = pd.read_csv('dataset-hasil-augmented/fear_noDup.csv', index_col=0)
fear_data = fear_data.reset_index(drop=True)
fear_data = fear_data.rename({'Text': 'text'}, axis=1)
fear_data = fear_data["text"].str[12:].replace("<","").replace(">","")
fear_data = fear_data.to_frame()

In [ ]:
%%time
fear_data["emotion"] = fear_data["text"].apply(get_emotion)

In [ ]:
fear_data